In [1]:
import pandas as pd
import numpy as np

In [5]:
ttc = pd.read_csv('ttc_delays_cleaned.csv')

In [7]:
ttc_stations = ttc['Station']
ttc_stations.head(2)

0    HIGH PARK
1     SHEPPARD
Name: Station, dtype: object

In [15]:
stations = pd.read_csv('csv_originals/station_ridership_grade.csv')
stations['Station'] = stations['Station'].str.upper()
stations.head(2)

,Station,Grade,2015 ridership
0,COLLEGE,Underground,"47,790"
1,DAVISVILLE,Surface,"25,330"


In [17]:
len(ttc)

76801

In [35]:
station_index = stations.set_index('Station')
station_index.index

Index([           'COLLEGE',         'DAVISVILLE',             'DUNDAS',
                   'DUPONT',           'EGLINTON',      'EGLINTON WEST',
                    'FINCH',          'GLENCAIRN',               'KING',
                 'LAWRENCE',      'LAWRENCE WEST',             'MUSEUM',
        'NORTH YORK CENTRE',            'OSGOODE',              'QUEEN',
             'QUEEN'S PARK',           'ROSEDALE',      'SHEPPARD WEST',
               'ST. ANDREW',          'ST. CLAIR',     'ST. CLAIR WEST',
              'ST. PATRICK',         'SUMMERHILL',              'UNION',
                'WELLESLEY',             'WILSON',         'YORK MILLS',
                 'YORKDALE',        'BLOOR–YONGE',            'SPADINA',
               'ST. GEORGE',     'SHEPPARD–YONGE',                  nan,
                 'BATHURST',                'BAY',          'BROADVIEW',
             'CASTLE FRANK',            'CHESTER',           'CHRISTIE',
                  'COXWELL',           'DONLANDS', 

In [44]:
ttc_stations_index = pd.Index(ttc_stations.unique())
ttc_stations_index

Index([         'HIGH PARK',           'SHEPPARD',          'LANSDOWNE',
                    'BLOOR',           'DUFFERIN',  'NORTH YORK CENTRE',
                'RUNNYMEDE',              'QUEEN',         'ST. ANDREW',
                'WELLESLEY',            'KIPLING',        'ST. PATRICK',
                  'KENNEDY',               'JANE',           'OLD MILL',
                   'WILSON',          'DOWNSVIEW',                'BAY',
                    'FINCH',            'COXWELL',     'ST. CLAIR WEST',
                   'DUPONT',         'YORK MILLS',        'DUNDAS WEST',
                  'OSGOODE',            'COLLEGE',              'YONGE',
            'VICTORIA PARK',         'ST. GEORGE',          'BROADVIEW',
                 'EGLINTON',           'ROSEDALE',               'PAPE',
                'GREENWOOD',          'ISLINGTON',             'WARDEN',
               'DAVISVILLE',           'LAWRENCE',      'EGLINTON WEST',
                    'KEELE',              'UNION', 

Lets find which of the Stations don't match

In [49]:
station_index.index.difference(ttc_stations_index)

Index(['BLOOR–YONGE', 'MAIN STREET', 'SHEPPARD WEST', 'SHEPPARD–YONGE'], dtype='object')

In [50]:
ttc_stations_index.difference(station_index.index)

Index(['BLOOR', 'DOWNSVIEW', 'MAIN', 'SHEPPARD', 'YONGE'], dtype='object')

In [ ]:
Looks like we need to fix:
BLOOR–YONGE --> Bloor and Yonge?
MAIN STREET --> MAIN
SHEPPARD WEST --> SHEPPARD
SHEPPARD–YONGE --> DOWNSVIE

In [58]:
# Remove the STREET from main street like we did in the other file
stations.loc[stations['Station'] == 'MAIN STREET'] = 'MAIN'

# Rename Sheppard West to DOWNSVIEW because this data predates 2017
stations.loc[stations['Station'] == 'SHEPPARD WEST'] = 'DOWNSVIEW'

In [59]:
station_index = stations.set_index('Station')
ttc_stations_index = pd.Index(ttc_stations.unique())
station_index.index.difference(ttc_stations_index)

Index(['BLOOR–YONGE', 'SHEPPARD–YONGE'], dtype='object')

In [60]:
ttc_stations_index.difference(station_index.index)

Index(['BLOOR', 'SHEPPARD', 'YONGE'], dtype='object')